I thought it would be interesting to experiment with creating an image that changed each time it was drawn.
Now this isn't the most exciting realisation of that idea, but it does also touch on an idea I want to look at
developing further as part of `arlunio` itself. Applying the concept of interfaces to arlunio images I think
could be very powerful - imagine having some generic "human" that knew how to walk and all you had to
do was tell it how to draw one...

Anyway back to this image, as you've probably noticed by now it's a clock - not a very interesting clock
but it tells the time, in fact it's frozen at the time that it was when the image was rendered.

In [ ]:
import arlunio as ar
import numpy as np

from arlunio import Shapes as S
from datetime import datetime

As the clock is made up from a collection of distinct shapes you need to ensure that each
component needs to be defined with a consistent scale otherwise it won't look right. I found
it useful to say that the radius of the clockface is `1` and then define everything else relative
to that. However since the default scale of the grid an image is defined on is also `1` the
clock would look a bit cramped as it would run right up againt the edge.

Rather than work to a more awkard scale I'm defining a `scale` paramaeter that I'm going to pass
to each shape. This changes the size of the underlying cartesian grid that is mapped onto the
image when drawn. By enlarging this grid each shape will appear smaller and fit better into the
final image

In [ ]:
scale = 1.1

As you've probably noticed the hands of the clock are just rectangles, but instead of
using the built in `Rectangle` shape in the standard library I've decided to define
a custom `ClockHand` shape. This is because the standard `Rectangle` shape is hard to
control in this scenario (perhaps this should be improved 🤔...) You will also see that 
I'm using [polar coordinates](https://en.wikipedia.org/wiki/Polar_coordinate_system)
instead of the conventional \\(x\\) and \\(y\\)

By defining the `ClockHand` shape with polar coordinates adding the ability to set the
rotation becomes easy since all we have to do is take an offset `t0` and subtract it from
the angle parameter `t`. Something to note is that when `arlunio` maps polar coordinates onto
an image it follows the mathematical convention of having the line `t = 0` be at
3 o'clock, so in order to make positioning the hands is easier later on I also shift this
coordinate by \\(\frac{\pi}{2}\\) (90 degrees) so when `t0 = 0` the hand will be pointing at
12 o'clock

Unfortunately while nice for expressing rotations, polar coordinates are certainly not the
best at expressing a rectangle. Thankfully the conversion from polar to standard \\(x\\) and
\\(y\\) coordinates is quite straightforward

\begin{align}
    x &= r\cos{(t)} \\\\
    y &= r\sin{(t)} 
\end{align}

Once we have our \\(x\\) and \\(y\\) coordinates it's easy enough to define a rectangle as
(ar.)all the \\(x\\) values between `0` and the `length` **and** all the \\(y\\) values
between `-size` and `size`.

In [ ]:
@ar.shape
def ClockHand(r, t, *, length=1, size=0.025, t0=0):
    t -= np.pi/2 - t0
    
    x = r * np.cos(t)
    y = r * np.sin(t)
      
    return ar.all(
        x > 0, x < length,
        np.abs(y) < size
    )

Similar to...

In [ ]:

@ar.shape
def Numeral(r, t, *, t0=0):
    t -= np.pi/2 - t0
    
    x = r * np.cos(t)
    y = r * np.sin(t)
    
    return ar.all(
        x > 0.8, x < 0.95,
        np.abs(y) < (0.1 * x) - 0.07
    )

In [ ]:
def make_clock(dt, hour_hand=None, minute_hand=None, second_hand=None):
    hour = dt.hour if dt.hour <= 12 else dt.hour - 12
    minutes = dt.minute
    seconds = dt.second
    
    second_hand.t0 = 2 * np.pi * (seconds / 60) 
    minute_hand.t0 = 2 * np.pi * (minutes / 60)
    hour_hand.t0 = 2 * np.pi * ((hour / 12) + (minutes / 600))
    
    center = S.Circle(r=0.2)
    
    clock = hour_hand + minute_hand + second_hand + center
    return clock

In [ ]:
hour_hand = ClockHand(size=0.02, length=0.5, scale=scale)
minute_hand = ClockHand(size=0.02, length=0.8, scale=scale)
second_hand = ClockHand(size=0.01, length=.95, color="ff0000", scale=scale)

clock = S.Circle(r=1, pt=0.01, scale=scale)

for i in range(12):
    t0 = i * (np.pi / 6)
    clock += Numeral(t0=t0, scale=scale)

clock += make_clock(datetime.now(), hour_hand, minute_hand, second_hand)